<a href="https://colab.research.google.com/github/bhandiaRE/Python-Web-Scraping-/blob/main/PySpark/PySpark_log.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip3 install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 53 kB/s 
     |████████████████████████████████| 199 kB 74.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=6b7a5cf77d5df645c35c361be0b5deaa01b2f37e159d7172429632d6eeb7d437
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [2]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().setAppName("Pyspark_Log").setMaster("local[*]")
sc = SparkContext(conf=conf)
print(sc)
print("ready to go")

<SparkContext master=local[*] appName=Pyspark_Log>
ready to go


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
rdd = sc.textFile("/content/drive/MyDrive/log.txt")

In [6]:
rdd = rdd.repartition(4)
print(sc.defaultParallelism)
print(rdd.getNumPartitions())

2
4


In [36]:
print("The number of records is",rdd.count())
rdd.takeSample(False, 20, 1234)

The number of records is 62


['03/22 08:51:02 INFO   :..reg_process: registering process with the system',
 '03/22 08:51:06 INFO   :.....mailslot_create: creating mailslot for RSVP via UDP',
 ' 03 ',
 '03/22 08:51:06 TRACE  :..entity_initialize: interface 9.37.65.139, entity for rsvp allocated and ',
 '03/22 08:51:06 INFO   :...mailbox_register: mailbox allocated for timer',
 '03/22 08:51:06 TRACE  :..entity_initialize: interface 129.1.1.1, entity for rsvp allocated and initialized',
 '03/22 08:51:06 INFO   :....mailbox_register: mailbox allocated for rsvp-udp',
 '03/22 08:51:06 INFO   :.....mailslot_create: creating mailslot for RSVP via UDP',
 '03/22 08:51:06 INFO   :....mailbox_register: mailbox allocated for rsvp-udp',
 '03/22 08:51:06 INFO   :...read_physical_netif: index #6, interface LOOPBACK has address 127.0.0.1, ifidx 0',
 '03/22 08:51:06 TRACE  :...read_physical_netif: Home list entries returned = 7',
 '03/22 08:51:06 INFO   :...read_physical_netif: index #0, interface VLINK1 has address 129.1.1.1, ifid

**Lines that have UDP**

In [34]:
import re
def collect_word(line):
  return re.compile('\w+').findall(line.lower())

print(collect_word('we are TESTING !  ? oK?'))

['we', 'are', 'testing', 'ok']


In [35]:
rdd_UDP = rdd.filter(lambda line: "warning" in collect_word(line))

rdd_UDP.collect()

['03/22 08:51:06 WARNING:.....mailslot_create: setsockopt(MCAST_ADD) failed - EDC8116I Address not available.',
 '03/22 08:51:06 WARNING:.....mailslot_create: setsockopt(MCAST_ADD) failed - EDC8116I Address not available.',
 '03/22 08:51:06 WARNING:.....mailslot_create: setsockopt(MCAST_ADD) failed - EDC8116I Address not available.',
 '03/22 08:51:06 WARNING:.....mailslot_create: setsockopt(MCAST_ADD) failed - EDC8116I Address not available.']

**Get lines with IP Address and INFO**

In [10]:
def get_IP(line):
  return re.findall(r'[0-9]+.[0-9]+.[0-9]+.[0-9]+.', line)

get_IP('read_physical_netif: index #6, interface LOOPBACK has address 127.0.0.1, ifidx 0')

['127.0.0.1,']

In [11]:
#rdd.filter(lambda line: line.split(',')[0])\
rdd.filter(lambda line: "info" in collect_word(line))\
  .filter(lambda line: len(get_IP(line))>1).collect()


['03/22 08:51:06 INFO   :...read_physical_netif: index #0, interface VLINK1 has address 129.1.1.1, ifidx 0',
 '03/22 08:51:06 INFO   :...read_physical_netif: index #1, interface TR1 has address 9.37.65.139, ifidx 1',
 '03/22 08:51:06 INFO   :...read_physical_netif: index #2, interface LINK11 has address 9.67.100.1, ifidx 2',
 '03/22 08:51:06 INFO   :...read_physical_netif: index #3, interface LINK12 has address 9.67.101.1, ifidx 3',
 '03/22 08:51:06 INFO   :...read_physical_netif: index #4, interface CTCD0 has address 9.67.116.98, ifidx 4',
 '03/22 08:51:06 INFO   :...read_physical_netif: index #5, interface CTCD2 has address 9.67.117.98, ifidx 5',
 '03/22 08:51:06 INFO   :...read_physical_netif: index #6, interface LOOPBACK has address 127.0.0.1, ifidx 0']

**Finding ifidx Connections**

In [12]:
rdd_Failed = rdd.filter(lambda line: "ifidx" in collect_word(line))

rdd_split = rdd_Failed.map(lambda line: (line.split()[12], 1))

rdd_split.groupByKey(4).mapValues(sum).sortBy(lambda x: x[1], ascending=False).take(5)

[('0', 2), ('4', 1), ('3', 1), ('1', 1), ('2', 1)]

**Data Collection**

In [29]:
def get_RSVP(line):
  return re.compile( '\w+' ).findall(line.lower())

rdd.filter(lambda line: "for" in get_RSVP(line))\
  .map(lambda line: line.lower().split('for')[1])\
  .map(lambda line: (line, 1)).groupByKey(4).mapValues(sum)\
  .sortBy(lambda x: x[1], ascending=False).collect()


[(' rsvp', 12),
 (' rsvp via udp', 6),
 (' rsvp-udp', 6),
 (' rsvp allocated and ', 3),
 (' rsvp allocated and initialized', 3),
 (' timer', 2)]